In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login

login(token="hf_.....................") 

model_name = "VyDat/llama3-8b-vietnamese-multi"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

SYS_INSTRUCT = """
Bạn là một trợ lý AI thông minh, có khả năng trả lời các câu hỏi bằng tiếng Việt một cách thân thiện và tự nhiên. 
Nếu câu hỏi liên quan đến ngôn ngữ ký hiệu Việt Nam (VSL), hãy chuyển đổi hoặc giải thích ký hiệu sang ngôn ngữ nói một cách chính xác. 
Cung cấp câu trả lời ngắn gọn, hữu ích và phù hợp với ngữ cảnh của câu hỏi.
"""

def format_prompt(user_input):
    messages = [
        {"role": "system", "content": SYS_INSTRUCT},
        {"role": "user", "content": user_input}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Vòng lặp inference
def main():
    while True:
        user_input = input()
        if user_input.lower() == 'quit':
            print('Tạm biệt\n')
            break

        prompt = format_prompt(user_input)

        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=False
        )

        # Cắt phần prompt để chỉ lấy phần sinh mới
        generated_ids = outputs.sequences[0][inputs["input_ids"].shape[-1]:]
        assistant_response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

        print(assistant_response)
        print('\n\n')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 tôi màu đỏ thích


Bạn đang nói về màu sắc. "Màu đỏ" là một màu sắc cơ bản trong hệ màu RGB. Màu đỏ thường được mô tả là màu có bước sóng dài nhất trong phổ ánh sáng nhìn thấy được của con người.





 chuyển thành ngôn ngữ nói: "tôi màu đỏ thích"


Tôi thích màu đỏ.





 bạn biết gì về ngôn ngữ kí hiệu VSL


Ngôn ngữ ký hiệu Việt Nam (VSL) hay còn được gọi là ngôn ngữ ký hiệu Việt Nam hoặc ngôn ngữ ký hiệu Việt Nam quốc gia, là ngôn ngữ ký hiệu được sử dụng tại Việt Nam. Ngôn ngữ ký hiệu này được sử dụng để giao tiếp với người khiếm hoặc người có khó khăn trong việc nói và nghe.

Ngôn ngữ ký hiệu Việt Nam dựa trên hệ thống ký hiệu Việt Nam, bao gồm hơn 600 ký hiệu khác nhau. Hệ thống này được thiết kế để thể hiện các âm thanh và từ trong ngôn ngữ Việt Nam.

VSL được sử dụng trong nhiều lĩnh vực như giáo dục, y tế, thương mại và giao tiếp hàng ngày. Ngoài ra, ngôn ngữ ký hiệu Việt Nam còn được sử dụng trong các hoạt động thể thao và giải trí.

Một số đặc điểm của ngôn ngữ ký hiệu VSL bao gồm:

- Hệ thống ký hiệu đa dạng và phong phú
- Có thể được sử dụng độc lập hoặc kết hợp với ngôn ngữ nói
- Có thể được sử dụng để giao tiếp với người khiếm hoặc người có khó khăn trong việc nói và nghe
- Có thể được sử dụng trong nhiều lĩnh vực khác nhau





KeyboardInterrupt: Interrupted by user